# S10 — Proxecto final (I): proposta + prototipo mínimo
**Curso:** Programación Musical con Python — 2º Trimestre  
**Duración:** 1 sesión (≈ 1 hora)

## 🎯 Obxectivos
- Escoller un tema de proxecto (simbólico / audio / exploración).
- Definir unha **proposta clara**: que vas facer, con que datos e por que.
- Construír un **prototipo mínimo** que xa funcione (MVP).
- Deixar o notebook preparado para continuar en S11 e rematar en S12.

> ✅ Hoxe non buscamos perfección: buscamos un **primeiro prototipo funcional**.


## 0) O proxecto final: formatos permitidos

### 🎼 Opción A — Simbólico (music21)
- xerador musical (regras)
- variacións sobre unha melodía
- análise dunha obra sinxela (intervalos, ritmos, patróns)
- acompañamento automático básico

### 🎧 Opción B — Audio (librosa)
- análise de interpretacións (tempo, ataques, enerxía)
- comparación de instrumentos/estilos (features)
- estudo de fragmentos dunha gravación

### 🤖 Opción C — Exploración (suave)
- clasificador por regras + features
- preparación de datos (carpetas, CSV, features)
- conexión conceptual con live coding (non obrigatorio)

> Escoller **unha** opción. Máximo 2 ideas se están conectadas.


## 1) Checklist dunha boa proposta (10 min)

A proposta debe responder:

1) **Título** (curto e musical)  
2) **Pregunta/obxectivo** (que queres demostrar ou crear?)  
3) **Datos** (que melodías/audios vas usar?)  
4) **Ferramentas** (music21/librosa/numpy/matplotlib)  
5) **Plan en 3 pasos** (moi concreto)  
6) **Entrega final** (que se verá/escuchará?)  
7) **Riscos** (que pode fallar?) e plan B

➡️ Escribe isto en Markdown no notebook (copiando a plantilla).


### ✍️ Plantilla de proposta (copia e enche)

**Título:**  
**Opción (A/B/C):**  
**Obxectivo:**  
**Datos:**  
**Ferramentas:**  
**Plan (3 pasos):**  
1)  
2)  
3)  
**Entrega final:**  
**Riscos + plan B:**


## 2) Prototipo mínimo (MVP) — elixe unha das 3 rutas

O MVP debe cumprir:
- Executa sen erros
- Produce **un output**: MIDI/XML, gráfica, ficheiro WAV, comparación, etc.
- Ten código modular (funcións pequenas)
- Ten comentarios por sección (ler como partitura)

A continuación tes 3 prototipos-base. Elixe un e modifícao.


## A) MVP Simbólico (music21): xerador + exportación

Crea unha frase e xera variacións:
- transposición
- variación rítmica
- estrutura A–A2–B–A


In [ ]:
# ✅ MVP A (music21)
!pip -q install music21
from music21 import stream, note, meter, tempo, key

def crear_base(time_sig="4/4", bpm=100, tonalidade="C"):
    s = stream.Stream()
    s.append(meter.TimeSignature(time_sig))
    s.append(tempo.MetronomeMark(number=bpm))
    s.append(key.Key(tonalidade))
    return s

def frase(alturas, ritmos):
    s = stream.Stream()
    for a, d in zip(alturas, ritmos):
        if a == "REST":
            s.append(note.Rest(quarterLength=d))
        else:
            s.append(note.Note(a, quarterLength=d))
    return s

def transpoñer_alturas(alturas, semitonos):
    # usando music21 (máis fiable que o converter didáctico)
    out = []
    for a in alturas:
        if a == "REST":
            out.append(a)
        else:
            n = note.Note(a)
            n.transpose(semitonos, inPlace=True)
            out.append(n.nameWithOctave)
    return out

# Datos base
A_alt = ["C4","D4","E4","G4","E4","D4","C4","REST"]
A_rit = [1,1,1,1,1,1,1,1]

# Variación: transposición +2 e ritmo distinto
A2_alt = transpoñer_alturas(A_alt, 2)
A2_rit = [0.5,0.5,1, 0.5,0.5,1, 2,2]  # suma 8

# Sección B (contraste)
B_alt = ["E4","F4","G4","A4","G4","F4","E4","D4"]
B_rit = [1]*8

# Montaxe
score = crear_base(bpm=110, tonalidade="C")
for sec in [frase(A_alt, A_rit), frase(A_alt, A_rit), frase(B_alt, B_rit), frase(A2_alt, A2_rit)]:
    for el in sec.recurse().notesAndRests:
        score.append(el)

score.show("text")

midi_path = score.write("midi")
xml_path = score.write("musicxml")
midi_path, xml_path


## B) MVP Audio (librosa): comparar dous audios por features + gráfica

Produce:
- waveform
- espectrograma
- fingerprint e distancia


In [ ]:
# ✅ MVP B (librosa)
!pip -q install librosa soundfile

import numpy as np
import matplotlib.pyplot as plt
import librosa, librosa.display
from IPython.display import Audio, display

def cargar_exemplo(nome):
    p = librosa.ex(nome)
    y, sr = librosa.load(p, sr=None)
    return y, sr

def fingerprint(y, sr):
    rms = librosa.feature.rms(y=y)[0]
    cent = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    zcr = librosa.feature.zero_crossing_rate(y)[0]
    return np.array([np.mean(rms), np.std(rms), np.mean(cent), np.std(cent), np.mean(zcr), np.std(zcr)], float)

def dist(a, b):
    return float(np.linalg.norm(a-b))

y1, sr1 = cargar_exemplo("trumpet")
y2, sr2 = cargar_exemplo("choice")

display(Audio(y1, rate=sr1))
display(Audio(y2, rate=sr2))

fp1 = fingerprint(y1, sr1)
fp2 = fingerprint(y2, sr2)
print("Distancia:", round(dist(fp1, fp2), 4))

# Gráfica: espectrograma do primeiro audio
S = np.abs(librosa.stft(y1))
S_db = librosa.amplitude_to_db(S, ref=np.max)

plt.figure()
librosa.display.specshow(S_db, sr=sr1, x_axis="time", y_axis="hz")
plt.colorbar(format="%+2.0f dB")
plt.title("Espectrograma (audio1)")
plt.show()


## C) MVP Exploración: “clasificador por regras” con features

Idea: decidir se un fragmento é “máis brillante” ou “máis escuro” só con centroide.


In [ ]:
# ✅ MVP C (regras)
!pip -q install librosa

import numpy as np
import librosa

def clasificar_brillo(y, sr, threshold=2000):
    cent = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    cent_m = float(np.mean(cent))
    etiqueta = "BRILLANTE" if cent_m > threshold else "ESCURO"
    return etiqueta, cent_m

y, sr = librosa.load(librosa.ex("trumpet"), sr=None)
lab, cent_m = clasificar_brillo(y, sr)
print("Etiqueta:", lab, "| centroide medio:", round(cent_m, 1))


## 3) Entregable da sesión S10

### O que tes que deixar feito hoxe (avaliable)
✅ Proposta completa (plantilla enchida)  
✅ Un MVP executado (A ou B ou C)  
✅ Un output gardado / mostrado (MIDI/XML, gráfica, número, etc.)  
✅ Notebook organizado por seccións con comentarios

> Isto conta como **10% do proxecto final** (proposta + prototipo).


## 🧠 Ler código como partitura (checkpoint)

No teu notebook final:
- `# PROPOSTA`
- `# DATOS`
- `# FUNCIÓNS`
- `# PROTOTIPO (MVP)`
- `# RESULTADOS`
- `# PRÓXIMOS PASOS (S11)`


In [ ]:
# ✅ Espazo para escribir “Próximos pasos”
# 1)
# 2)
# 3)
